## Loading and Indexing

In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("../../data/").load_data(show_progress=True)

Loading files: 100%|██████████| 1/1 [00:00<00:00,  5.23file/s]


In [2]:
from llama_index.core import VectorStoreIndex

In [3]:
import os
os.environ["OPENAI_API_KEY"] = input("Enter")

In [4]:
index = VectorStoreIndex.from_documents(documents,show_progress=True)

/Users/admin/miniconda3/envs/verloopAssignment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 22/22 [00:01<00:00, 14.48it/s]


In [23]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=100, chunk_overlap=10)
# global
# from llama_index.core import Settings
# Settings.text_splitter = text_splitter

In [24]:
index = VectorStoreIndex.from_documents(documents,show_progress=True,transformations=[text_splitter])

Generating embeddings: 100%|██████████| 319/319 [00:07<00:00, 43.58it/s]


## Storing

In [25]:
index.storage_context.persist("pre_computed_embeddings")

### Instead of storing the embedding into a directory, we will now store it in a vector store (Chroma).

In [5]:
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext


In [6]:

# initialize client, setting path to save data
db = chromadb.PersistentClient()
# create collection
chroma_collection = db.get_or_create_collection("quickstart")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create your index
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,show_progress=True
)


Generating embeddings: 100%|██████████| 22/22 [00:01<00:00, 15.58it/s]


## Quering.

In [8]:
query_engine = index.as_query_engine()
response = query_engine.query("What is the meaning of life?")
print(response)

The meaning of life is a complex and subjective concept that has been pondered by philosophers, theologians, and individuals throughout history. It often involves questions about existence, purpose, fulfillment, and the significance of one's actions and experiences. Different people and belief systems may offer varying perspectives on the meaning of life, leading to a wide range of interpretations and understandings.


In [9]:
from llama_index.core import  get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

# query
response = query_engine.query("What did the author do growing up?")
print(response)

Empty Response
